# FLAG multipulses

It checks that SIRENA reconstructed photons are correctly labelled based on ELOWRES vs. SIGNAL and AVG4SD vs. SIGNAL confidence contours.   

For each SIRENA photon, it answers these questions:   
1. Is it a single PRIMARY (dist_to_prev > 1563) BUT it would be flagged as PILED-UP (outside confidence intervals)?   
2. Is is a PILED-UP pulse (in the table of missing or bad-reconstructed photons) BUT it is flagged as SINGLE?   

Photons not considered (not analysed):
* Solitary Pulses that do not have a close previous or posterior partner (< 100 samples). These pulses have not been even simulated in `xifusim` to save resources.
* Pulses reconstructed with OF-length = 8 samples (same as ELOWRES)   
* Pulses classified as SECONDARIES: previous pulse closer than 1563 samples -> diagnostic confidence intervals do not work for them   

PROCEDURE:   
1. Import modules   
2. Read parameters:   
    - location of simulated monochromatic files for diagnostic confidence intervals   
    - width of conf. interval (n_sigmas)   
    - order of the polynomial for the confidence region fit   
    - secondaries definition
3. Confidence areas definition:   
    3.1 Read/Reconstruct monochromatic HR pulses with SIRENA using all the pre-defined filter lengths: get ELOWRES and AVG4SD columns   
    3.2 For each filter length: ELOWRES vs Reconstructed_Calibration_Energy (SIGNAL) -> polynomial fit to confidence region   
    3.3 For each filter length: AVG4SD vs Reconstructed_Calibration_Energy (SIGNAL) -> polynomial fit to confidence region   
4. Run over all the photons in SIRENA files (all simulations for a given flux) and check if there are miss-flagged photons. Add these miss-classifications to a list (for plotting afterwards)   
5. Plot:   
    5.1 confidence intervals for monochromatic simulated pulses (all filter lengths) and residuals plots (data points outside confidence regions)   
    5.2 miss-flagged photons identified in step 4   

   

### Import modules

In [ ]:
import ipywidgets as widgets 
%matplotlib widget

import os
import glob
from subprocess import run, PIPE, STDOUT
import tempfile
from astropy.io import fits
import ast
import pandas as pd
import numpy as np
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import xml.etree.ElementTree as ET
import auxiliary as aux
from auxiliary import get_max_photons, get_polyfit_intervals_columns


In [ ]:
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = f"{tmpDir}:{os.environ['PFILES']}"
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"
SIXTE = os.environ["SIXTE"]

### Read parameters

In [ ]:
# read xifusim simulated files
#datadir="/dataj6/ceballos/INSTRUMEN/EURECA/ERESOL/CEASaclay/October2024"
datadir="/dataj6SIRENA/ceballos/INSTRUMEN/EURECA/TN350_detection/2024_revision/singles"
simfiles = glob.glob(f"{datadir}/mono*keV_5000p_50x30.fits")
secondary_samples = 1563
verbose = 1
simEnergies = (0.2, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0)
nsigmas = 5 # for confidence interval polyfit
poly_order = 8 # order of polynomial fit

## Create confidence contours in diagnostic plots

### Read/create SIRENA monochromatic files   

Read/Reconstruct monochromatic `xifusim` simulated files using all filter lengths in XML file

In [ ]:
lib_sirena = f"{datadir}/optimal_filters_6keV_50x30.fits"
assert os.path.exists(lib_sirena), f"{lib_sirena} does not exist"
xml_xifusim = f"{datadir}/config_xifu_50x30_v3_20240917.xml"
assert os.path.exists(xml_xifusim), f"{xml_xifusim} does not exist"
# read filter lengths from XML file: under <reconstruction> tag in filtlen attribute
# Load and parse the XML file
tree = ET.parse(xml_xifusim)
root = tree.getroot()
# Extract all values of "filtlen" attributes
filtlen_values = [elem.attrib["filtlen"] for elem in root.iter() if "filtlen" in elem.attrib]
# convert it to a list of integers
filtlen_values = [int(x) for x in filtlen_values]

In [ ]:
# reconstruct simulated files with SIRENA
for oflen in filtlen_values:
    for xifusimfile in simfiles:
        assert os.path.exists(xifusimfile), f"{xifusimfile} does not exist"
        # remove path from filename:
        filename = os.path.basename(xifusimfile)
        # get energy value from xifusim file name
        energy = float(filename.split("_")[0].replace("keV", "").replace("mono", ""))
        reconsfile = f"{datadir}/events_mono{energy}keV_5000p_50x30_of{oflen}.fits"
        if not os.path.exists(reconsfile):
            comm = (f"tesrecons"
                    f" Recordfile={xifusimfile}"
                    f" TesEventFile={reconsfile}"
                    f" LibraryFile={lib_sirena}"
                    f" XMLFile={xml_xifusim}"
                    f" clobber=yes"
                    f" EnergyMethod=OPTFILT"
                    f" OFStrategy=FIXED"
                    f" OFLength={oflen}"
                    f" filtEeV=6000"
                    f" OFNoise=NSD"
            )
            aux.vprint(f"Doing reconstruction for xifusim file {xifusimfile}")
            aux.vprint(f"Running {comm}")
            
            output_tesrecons = run(comm, shell=True, capture_output=True)
            assert output_tesrecons.returncode == 0, f"tesrecons failed to run => {output_tesrecons.stderr.decode()}"
            assert os.path.exists(reconsfile), f"tesrecons did not produce an output file"
        else:
            aux.vprint(f"Reconstructed file {reconsfile} already exists")
reconsfiles = glob.glob(f"singles/events_mono*keV_5000p_50x30_*.fits")
max_photons = get_max_photons(reconsfiles)        

### Store information of SIRENA reconstruction of monochromatic files   

Save info of columns SIGNAL, ELOWRES and AVG4SD and take also median and stdev values     

In [ ]:

# create numpy arrays to store the data
# the first dimension is the energy, the second dimension is the number of photons  
# inititalize the arrays with NaN
SIGNAL_mono = np.full((len(simEnergies),len(filtlen_values),max_photons), np.nan)
ELOWRES_mono = np.full((len(simEnergies), len(filtlen_values),max_photons), np.nan)
AVG4SD_mono = np.full((len(simEnergies), len(filtlen_values),max_photons), np.nan)
medianSIGNAL = np.full((len(simEnergies), len(filtlen_values)), np.nan)
medianELOWRES = np.full((len(simEnergies), len(filtlen_values)), np.nan)
stdELOWRES = np.full((len(simEnergies), len(filtlen_values)), np.nan)
medianAVG4SD = np.full((len(simEnergies), len(filtlen_values)), np.nan)
stdAVG4SD = np.full((len(simEnergies), len(filtlen_values)), np.nan)

#initialize array to store simulated energies in integer format
simEnergies_lab = np.array(simEnergies, dtype='str')

for ifl in range(len(filtlen_values)):
    ofl = filtlen_values[ifl]
    for ie in range(len(simEnergies)):
        simE = simEnergies[ie]
        if simE >= 1:
            simEnergies_lab[ie] = int(simEnergies[ie])
        sirena_file = f"{datadir}/events_mono{simE}keV_5000p_50x30_of{ofl}.fits"
        #print(f"Reading {sirena_file}")
        f = fits.open(sirena_file)
        # read the data and store 
        # store the array in the SIGNAL column in the second dimension of the SIGNAL array
        signal_data = f[1].data['SIGNAL']
        elow_data = f[1].data['ELOWRES']
        avg4sd_data = f[1].data['AVG4SD']
        SIGNAL_mono[ie, ifl, :len(signal_data)] = signal_data
        ELOWRES_mono[ie, ifl, :len(elow_data)] = elow_data
        AVG4SD_mono[ie, ifl, :len(avg4sd_data)] = avg4sd_data
        medianSIGNAL[ie, ifl] = np.nanmedian(signal_data)
        medianELOWRES[ie, ifl] = np.nanmedian(elow_data)
        stdELOWRES[ie, ifl] = np.nanstd(elow_data)
        medianAVG4SD[ie, ifl] = np.nanmedian(avg4sd_data)
        stdAVG4SD[ie, ifl] = np.nanstd(avg4sd_data)
        f.close()
    

### Polynomial fit to median+n_sigmas*stdev    

Create these topt & bottom polynomials for all the filter lengths

In [ ]:
# foreach filter length fit polynomials to create confidence intervals
# initialize arrays to store the polynomial coefficients for the confidence intervals
poly_top_coeffs_ELOWRES = np.full((len(filtlen_values), poly_order+1), np.nan)
poly_bottom_coeffs_ELOWRES = np.full((len(filtlen_values), poly_order+1), np.nan)
poly_top_coeffs_AVG4SD = np.full((len(filtlen_values), poly_order+1), np.nan)
poly_bottom_coeffs_AVG4SD = np.full((len(filtlen_values), poly_order+1), np.nan)

for ifl in range(len(filtlen_values)):
    poly_coeffs_dict = get_polyfit_intervals_columns(columnX=SIGNAL_mono[:,ifl,:], columnY=ELOWRES_mono[:,ifl,:], nsigmas=nsigmas, order=poly_order)
    poly_top_coeffs_ELOWRES[ifl] = poly_coeffs_dict['top']
    poly_bottom_coeffs_ELOWRES[ifl] = poly_coeffs_dict['bottom']

    poly_coeffs_dict = get_polyfit_intervals_columns(columnX=SIGNAL_mono[:,ifl,:], columnY=AVG4SD_mono[:,ifl,:], nsigmas=nsigmas, order=poly_order)
    poly_top_coeffs_AVG4SD[ifl] = poly_coeffs_dict['top']
    poly_bottom_coeffs_AVG4SD[ifl] = poly_coeffs_dict['bottom']

## Check flagging of SIRENA photons

For each SIRENA reconstructed photons (only close photons have been simulated and reconstructed), check if:   
* Single photons are flagged as 'singles'   
* Piled-up photons ('bad-reconstructed' or 'Non-reconstructed') are flagged as piled-up       

In [ ]:
model = "crab"
flux_mcrab_str = "0.50"
exposure = 4331
fluxdir = f"/dataj6/ceballos/INSTRUMEN/EURECA/TN350_detection/2024_revision/flux{flux_mcrab_str}mcrab"

# create a structure to save SIGNAL, ELOWRES and AVG4SD of incorrectly flagged photons
bad_flagged = {"SIGNAL": [], "ELOWRES": [], "AVG4SD": []}

for isim in range(1,101):
    #if not isim == 1:
    #    continue
    aux.vprint(f"Processing sim_{isim} and flux{flux_mcrab_str}mcrab")
    filestring = f"{fluxdir}/sim_{isim}/{model}_flux{flux_mcrab_str}_Emin2_Emax10_exp{exposure}_RA0.0_Dec0.0_nofilt_infoc"
    # read CSV file
    csv_file = f"{fluxdir}/sim_{isim}/00_info_nofilt_infoc_sim{isim}_missing.csv"
    missing_table = pd.read_csv(csv_file, converters={"Non-reconstructed photons": ast.literal_eval,
                                                      "Bad-reconstructed photons": ast.literal_eval})
    
    # list of SIRENA files
    reconsfiles = glob.glob(f"{filestring}_pixel*_sirena.fits")
    for reconsfile in reconsfiles:
        #aux.vprint(f"Checking {reconsfile}")
        ipixel = int(reconsfile.split("_")[-2].replace("pixel", ""))
        #read piximpact
        piximpact_file = f"{filestring}_pixel{ipixel}_piximpact.fits"
        with fits.open(piximpact_file) as f:
            TIME_piximpact = f[1].data['TIME'].copy()
            PH_ID_piximpact = f[1].data['PH_ID'].copy()
        # read sirena file
        with fits.open(reconsfile) as f:
            TIME = f[1].data['TIME']
            SIGNAL = f[1].data['SIGNAL']
            ELOWRES = f[1].data['ELOWRES']
            AVG4SD = f[1].data['AVG4SD']
            PH_ID = f[1].data['PH_ID']
            GRADE1 = f[1].data['GRADE1']
            GRADE2 = f[1].data['GRADE2']
            # for each row in sirena file, check if the photon is inside the 5-sigma confidence interval
            for irow in range(len(SIGNAL)):
                time_irow = TIME[irow]
                signal_irow = SIGNAL[irow]
                elowres_irow = ELOWRES[irow]
                avg4sd_irow  = AVG4SD[irow]
                grade1_irow = GRADE1[irow]
                grade2_irow = GRADE2[irow]
                # if number of values != 0 in PH_ID[irow] is 1, then it is a single photon
                ph_nonzero_sequence = PH_ID[irow][np.nonzero(PH_ID[irow])]
                if len(ph_nonzero_sequence) == 1:
                    # single photon
                    #aux.vprint(f"For irow {irow} single photon: PH_ID[irow] = {PH_ID[irow][np.nonzero(PH_ID[irow])]}")
                    ph_id_irow = PH_ID[irow][0]
                else:
                    # more than one photon in the record: check corresponding time in piximpact file
                    min_time_diff = float('inf')
                    for ph_id in ph_nonzero_sequence:
                        time_ph_piximpact = TIME_piximpact[PH_ID_piximpact == ph_id]
                        time_diff = abs(time_ph_piximpact-time_irow)
                        if time_diff < min_time_diff:
                            min_time_diff = time_diff
                            ph_id_irow = ph_id
                    #aux.vprint(f"For irow {irow} ph_id_irow = {ph_id_irow}: ")
                # check if the photon is inside the 5-sigma confidence interval
                if grade1_irow == 8:
                    #aux.vprint(f"*** WARNING: Photon {ph_id_irow} for sim_{isim} pixel {ipixel} is ELOWRES: skipping")
                    continue
                if grade2_irow <= secondary_samples:
                    #aux.vprint(f"*** WARNING: Photon {ph_id_irow} for sim_{isim} pixel {ipixel} is SECONDARY: skipping")
                    continue               
                         
                # get the index value in filtlen_values that is closest to grade1_irow
                ifl = np.argmin(np.abs(filtlen_values-grade1_irow))
                # get the polynomial values for the SIGNAL value: choose the filter length that is closest to grade1_irow
                top_ELOWRES = poly.polyval(signal_irow, poly_top_coeffs_ELOWRES[ifl])
                bottom_ELOWRES = poly.polyval(signal_irow, poly_bottom_coeffs_ELOWRES[ifl])
                top_AVG4SD = poly.polyval(signal_irow, poly_top_coeffs_AVG4SD[ifl])
                bottom_AVG4SD = poly.polyval(signal_irow, poly_bottom_coeffs_AVG4SD[ifl])
                if (elowres_irow < top_ELOWRES and elowres_irow > bottom_ELOWRES and 
                    avg4sd_irow < top_AVG4SD and avg4sd_irow > bottom_AVG4SD): # single photon
                    # check if ph_id_irow is in the list of bad-reconstructed photons
                    flag_piledup = False
                else:
                    flag_piledup = True

                is_photon_in_bad_reconstructed = any(ph_id_irow in tabrow for tabrow in missing_table["Bad-reconstructed photons"])
                is_photon_in_missing = any(ph_id_irow in tabrow for tabrow in missing_table["Non-reconstructed photons"])
                wrong_flag = False
                if flag_piledup and not is_photon_in_bad_reconstructed and not is_photon_in_missing:
                    wrong_flag = True
                    aux.vprint(f"*** WARNING: Single photon {ph_id_irow} for sim_{isim} pixel {ipixel} would be flagged as a piled-up")
                elif not flag_piledup and (is_photon_in_bad_reconstructed or is_photon_in_missing):
                    wrong_flag = True
                    aux.vprint(f"*** WARNING: Bad-recons photon {ph_id_irow} for sim_{isim} pixel {ipixel} would be flagged as single")
                if wrong_flag:
                    aux.vprint(f"             elowres_irow = {elowres_irow}, top_ELOWRES = {top_ELOWRES}, bottom_ELOWRES = {bottom_ELOWRES}")
                    aux.vprint(f"             avg4sd_irow = {avg4sd_irow}, top_AVG4SD = {top_AVG4SD}, bottom_AVG4SD = {bottom_AVG4SD}")
                    # store the photon in the bad_flagged dictionary
                    # save signal, elowres, avg4sd to be added to the plot
                    bad_flagged["SIGNAL"].append(signal_irow)
                    bad_flagged["ELOWRES"].append(elowres_irow)
                    bad_flagged["AVG4SD"].append(avg4sd_irow)

## Plot confidence contours and miss-flagged photons   

Plot the data points from the simulated monochromatic pulses reconstructed with different filter lengths.   
Overplot the miss-flagged photons.   
Figure:   

|   ELOWRES vs SIGNAL                |          AVGD4SD vs SIGNAL     |
|----------------------------------- | ------------------------------ |
| Residuals (Points out of interval) | Residuals (Points out of interval) |

In [ ]:
plot_bad_flagged = True #if True, plot the flagged photons

fig, axes = plt.subplots(2, 2,  gridspec_kw={'height_ratios': [3, 1]}, figsize=(10, 6))
(ax1, ax2), (ax3, ax4) = axes

# ==========================
# PLOT 1-TOPL: ELOWRES vs SIGNAL
# ==========================
#ax1.set_xlabel("uncalibrated <SIGNAL> (~keV)")
ax1.set_ylabel("uncalibrated <ELOWRES> (~keV)")
ax1.set_xlim(0, 12.5)
ax1.set_ylim(0, 12.5)
ytop = ax1.get_ylim()[1]
xtoplot = np.linspace(0.1,13, 100)
# add a top title for the energy values
ax1.text(0.5*ax1.get_xlim()[1], 1.1*ytop, "Simulated Energy (keV)", ha='center', va='center', fontsize='small', color="darkgray")

for ifl in range(len(filtlen_values)):
    ofl = filtlen_values[ifl]
    # skip OFLEN=8 as it is equal to ELOWRES (not useful as diagnostic parameter)
    if ofl == 8:
        continue
    points_color = f"C{ifl}"
    for ie in range(len(simEnergies)):
        simE = simEnergies[ie]
        ax1.plot(SIGNAL_mono[ie, ifl], ELOWRES_mono[ie, ifl], marker='.', linestyle='None',markersize=3, color=points_color)
        # plot simulated energy vertical lines (only for first filter length)
        if ifl == 0:
            ax1.axvline(medianSIGNAL[ie, 0], color='gray', linestyle='--', alpha=0.1)
            # add labels where the vertical lines cross the top axis
            if not simEnergies[ie] == 0.5:
                ax1.text(medianSIGNAL[ie, 0], 13., simEnergies_lab[ie], ha='center', va='center', fontsize='small', color="darkgray")
            # add also small ticks at the top axis
            ax1.plot(medianSIGNAL[ie, 0], 12.5, alpha=0.5, marker='|', markersize=5,color="black")
        # plot median and the error bars
        ax1.errorbar(medianSIGNAL[ie, ifl], medianELOWRES[ie,ifl], yerr=nsigmas*stdELOWRES[ie, ifl], fmt='x', color='black', markersize=1)

    # plot the 5-sigma confidence interval
    # ax1.fill_between(medianSIGNAL[:, ifl], medianELOWRES[:,ifl]-nsigmas*stdELOWRES[:,ifl], 
    #                 medianELOWRES[:,ifl]+nsigmas*stdELOWRES[:,ifl], color='C0', alpha=0.2, label=f'{nsigmas}-sigma conf. int.')

    poly_top_plot = poly.polyval(xtoplot, poly_top_coeffs_ELOWRES[ifl])
    poly_bottom_plot = poly.polyval(xtoplot, poly_bottom_coeffs_ELOWRES[ifl])
    ax1.plot(xtoplot, poly_top_plot, linestyle='--', color=points_color, label=f'OFL: {ofl}')
    ax1.plot(xtoplot, poly_bottom_plot, linestyle='--', color=points_color)

# plot miss_flagged photons
if plot_bad_flagged:
    ax1.plot(bad_flagged["SIGNAL"], bad_flagged["ELOWRES"], marker='x', linestyle='None', markersize=3, 
             color='red', label=f'{len(bad_flagged["SIGNAL"])} wrong flagged phs')
# set legend for the 5-sigma confidence interval
ax1.legend(loc='lower right', fontsize='small')

#========================
# PLOT 2-TOPR: AVG4SD vs SIGNAL
#========================
ax2.set_xlim(0, 12.5)
ax2.set_ylim(-1, 550)
ytop = ax2.get_ylim()[1] 
ax2.set_ylabel("AVG4SD")
#ax2.set_xlabel("uncalibrated <SIGNAL> (~keV)")
# add a top title for the energy values
ax2.text(0.5*ax2.get_xlim()[1], ytop+50, "Simulated Energy (keV)", ha='center', va='center', fontsize='small', color="darkgray")

for ifl in range(len(filtlen_values)):
    ofl = filtlen_values[ifl]
    # skip OFLEN=8 as it is giving double blobs
    if ofl == 8:
        continue
    points_color = f"C{ifl}"
    for ie in range(len(simEnergies)):
        simE = simEnergies[ie]
        #reduce marker size to avoid overlap
        ax2.plot(SIGNAL_mono[ie, ifl], AVG4SD_mono[ie, ifl], marker='.', linestyle='None', markersize=2, color=points_color)
        # plot simulated energy vertical lines (only for first filter length)
        if ifl == 0:
            #set a color for the line
            ax2.axvline(medianSIGNAL[ie, 0], color='gray', linestyle='--', alpha=0.1)
            # add labels where the vertical lines cross the top axis
            if not simEnergies[ie] == 0.5:
                ax2.text(medianSIGNAL[ie, 0], ytop+15, simEnergies_lab[ie], ha='center', va='center', 
                         fontsize='small', color="darkgray")
            # add also small ticks at the top axis
            ax2.plot(medianSIGNAL[ie, 0], ytop, alpha=0.5, marker='|', markersize=5,color="black")
        # plot median and the error bars
        ax2.errorbar(medianSIGNAL[ie, ifl], medianAVG4SD[ie,ifl], yerr=nsigmas*stdELOWRES[ie, ifl], 
                     fmt='x', color='black', markersize=1)
    # plot the nsigmas-sigma confidence interval
    #ax2.fill_between(medianSIGNAL[:,ifl], medianAVG4SD[:,ifl]-nsigmas*stdAVG4SD[:,ifl], 
    #                medianAVG4SD[:,ifl]+nsigmas*stdAVG4SD[:,ifl], color=points_color, alpha=0.2, 
    #                label=f'{ofl} {nsigmas}-sigma conf. int.')
    
    poly_top_plot = poly.polyval(xtoplot, poly_top_coeffs_AVG4SD[ifl])
    poly_bottom_plot = poly.polyval(xtoplot, poly_bottom_coeffs_AVG4SD[ifl])
    ax2.plot(xtoplot, poly_top_plot, linestyle='--', color=points_color, label=f'OFL: {ofl}')
    ax2.plot(xtoplot, poly_bottom_plot, linestyle='--', color=points_color)

# set legend for the 5-sigma confidence interval: locate bottom right
ax2.legend(loc='lower right', fontsize='small')


# ==========================================
# PLOT 3-BOTTOML: ELOWRES residuals
# ==========================================
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))  # Forces scientific notation when needed
ax3.yaxis.set_major_formatter(formatter)
ax3.ticklabel_format(axis="y", style="sci", scilimits=(-3, 3))
ax3.yaxis.get_offset_text().set_fontsize(8) 

# calculate the distance of ELOWRES to the 5-sigma conf. int. boundary
min_dist = float('inf')
max_dist = float('-inf')
for ifl in range(len(filtlen_values)):
    ofl = filtlen_values[ifl]
    # skip OFLEN=8 as it is equal to ELOWRES (not useful as diagnostic parameter)
    if ofl == 8:
        continue
    # convert SIGNAL and ELOWRES to a 1D array
    SIGNAL1D = SIGNAL_mono[:,ifl,:].flatten()
    ELOWRES1D = ELOWRES_mono[:,ifl,:].flatten()
    dist = np.nan*np.ones_like(ELOWRES1D)
    poly_top_SIGNAL1D = poly.polyval(SIGNAL1D, poly_top_coeffs_ELOWRES[ifl])
    poly_bottom_SIGNAL1D = poly.polyval(SIGNAL1D, poly_bottom_coeffs_ELOWRES[ifl])
    for ii in range(len(SIGNAL1D)):
        dist_top = ELOWRES1D[ii] - poly_top_SIGNAL1D[ii]
        dist_bottom = ELOWRES1D[ii] - poly_bottom_SIGNAL1D[ii]
        if dist_top < 0 and dist_bottom > 0:
            dist[ii] = 0 # point is inside conf. int.
        elif dist_top > 0:
            dist[ii] = dist_top
        elif dist_bottom < 0:
            dist[ii] = dist_bottom
    if min(dist) < min_dist:
        min_dist = min(dist)
    if max(dist) > max_dist:
        max_dist = max(dist)

    # plot the distance
    ax3.plot(SIGNAL1D, dist, marker='.', linestyle='None', markersize=2)
ax3.set_xlim(0, 12.5)
ax3.set_ylim(1.1*min_dist, 1.2*max_dist)
ytop = ax3.get_ylim()[1]

# plot lines for simulated energy
for ie in range(len(simEnergies)):
    ax3.axvline(medianSIGNAL[ie, 0], color='gray',linestyle='--', alpha=0.1)
    # add also small ticks at the top axis
    ax3.plot(medianSIGNAL[ie,0], ytop, alpha=0.5, marker='|', markersize=5, color="black")

ax3.set_xlabel("uncalibrated <SIGNAL> (~keV)")
ax3.set_ylabel("ELOWRES - ConfInt")


# ==========================================
# PLOT 4-BOTTOMR: AVG4SD residuals
# ==========================================
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))  # Forces scientific notation when needed
ax4.yaxis.set_major_formatter(formatter)
ax4.ticklabel_format(axis="y", style="sci", scilimits=(-3, 3))
ax4.yaxis.get_offset_text().set_fontsize(8) 

# calculate the distance of AVG4SD to the 5-sigma conf. int. boundary
min_dist = float('inf')
max_dist = float('-inf')
for ifl in range(len(filtlen_values)):
    ofl = filtlen_values[ifl]
    # skip OFLEN=8 as it is giving double blobs
    if ofl == 8:
        continue
    # convert SIGNAL and ELOWRES to a 1D array
    SIGNAL1D = SIGNAL_mono[:,ifl,:].flatten()
    AVG4SD1D = AVG4SD_mono[:,ifl,:].flatten()
    dist = np.nan*np.ones_like(AVG4SD1D)
    poly_top_SIGNAL1D = poly.polyval(SIGNAL1D, poly_top_coeffs_AVG4SD[ifl])
    poly_bottom_SIGNAL1D = poly.polyval(SIGNAL1D, poly_bottom_coeffs_AVG4SD[ifl])
    for ii in range(len(SIGNAL1D)):
        dist_top = AVG4SD1D[ii] - poly_top_SIGNAL1D[ii]
        dist_bottom = AVG4SD1D[ii] - poly_bottom_SIGNAL1D[ii]
        if dist_top < 0 and dist_bottom > 0:
            dist[ii] = 0 # point is inside conf. int.
        elif dist_top > 0:
            dist[ii] = dist_top
        elif dist_bottom < 0:
            dist[ii] = dist_bottom
    if min(dist) < min_dist:
        min_dist = min(dist)
    if max(dist) > max_dist:
        max_dist = max(dist)
    # plot the distance
    ax4.plot(SIGNAL1D, dist, marker='.', linestyle='None', markersize=2)
ax4.set_xlim(0, 12.5)
ax4.set_ylim(1.1*min_dist, 1.2*max_dist)
ytop = ax4.get_ylim()[1]

# plot lines for simulated energy
for ie in range(len(simEnergies)):
    ax4.axvline(medianSIGNAL[ie,0], linestyle='--', color='gray', alpha=0.1)
    # add also small ticks at the top axis
    ax4.plot(medianSIGNAL[ie,0], ytop, alpha=0.5, marker='|', markersize=5, color="black")

ax4.set_xlabel("uncalibrated <SIGNAL> (~keV)")
ax4.set_ylabel("AVG4SD - ConfInt");

fig.tight_layout()